In [19]:
import os
from datetime import datetime, timedelta

import pandas as pd
from tvDatafeed import TvDatafeed, Interval

In [20]:
tv = TvDatafeed()

In [21]:
def pull_TVdata(ticker, exchange, interval, start_date, end_date):
    # Convert start_date and end_date from string to datetime objects
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    # Calculate the total time span in minutes
    time_difference = end_date - start_date
    
    # Determine the interval in minutes (modify based on the interval chosen)
    interval_map = {
        Interval.in_1_minute: 1,
        Interval.in_5_minute: 5,
        Interval.in_15_minute: 15,
        Interval.in_30_minute: 30,
        Interval.in_1_hour: 60,
        Interval.in_daily: 60 * 24,
        Interval.in_weekly: 60 * 24 * 7,
        Interval.in_monthly: 60 * 24 * 30  # Approximate for monthly data
    }
    
    # Get the interval in minutes from the map
    interval_in_minutes = interval_map.get(interval, 1)  # Default to 1 minute if not found

    # Calculate the number of bars needed
    total_minutes = time_difference.total_seconds() / 60  # Convert time difference to minutes
    n_bars = int(total_minutes / interval_in_minutes)

    # Pull data from TradingView
    df = tv.get_hist(symbol=ticker, exchange=exchange, interval=interval, n_bars=n_bars)
    
    return df

    

In [22]:
def process_TV_data(df):
    
    # Print column names with exact characters
    print("Original column names:", [f"'{col}'" for col in df.columns])
    
    # Check if 'symbol' exists in the DataFrame
    if 'symbol' not in df.columns:
        raise KeyError("'symbol' column is not found in the DataFrame.")
    
    # Extract the exchange and ticker from the symbol
    df['Exchange'] = df['symbol'].apply(lambda x: x.split(':')[0])
    df['Ticker'] = df['symbol'].apply(lambda x: x.split(':')[1])
    
    # Assuming underlying is the first two characters of the ticker (e.g., 'AL' in 'ALF2025')
    df['Underlying'] = df['Ticker'].apply(lambda x: x[:2])
    
    # Rename the 'datetime' column to 'Date'
    df = df.rename(columns={'datetime': 'Date'})
    
    # Verify the renaming step
    print("Column names after renaming:", df.columns)
    
    # Reorder the columns to match the required format
    final_columns = ['Date', 'Exchange', 'Ticker', 'Underlying', 'symbol', 'open', 'high', 'low', 'close', 'volume']
    processed_df = df[final_columns].copy()
    
    return processed_df


In [23]:
bp_df =  pull_TVdata(ticker="BP",exchange="NYSE",interval=Interval.in_daily,start_date="2000-01-01",end_date="2024-01-01")

In [24]:
shell_df = pull_TVdata(ticker="SHEL",exchange="NYSE",interval=Interval.in_daily,start_date="2000-01-01",end_date="2024-01-01")

In [25]:
oxy_df = pull_TVdata(ticker="OXY",exchange="NYSE",interval=Interval.in_daily,start_date="2000-01-01",end_date="2024-01-01")

In [26]:
cvx_df = pull_TVdata(ticker="CVX",exchange="NYSE",interval=Interval.in_daily,start_date="2000-01-01",end_date="2024-01-01")

In [27]:
xom_df = pull_TVdata(ticker="XOM",exchange="NYSE",interval=Interval.in_daily,start_date="2000-01-01",end_date="2024-01-01")

In [28]:
eqnr_df = pull_TVdata(ticker="EQNR",exchange="NYSE",interval=Interval.in_daily,start_date="2000-01-01",end_date="2024-01-01")

In [29]:
cl1_df = pull_TVdata(ticker="CL1!",exchange="NYMEX",interval=Interval.in_daily,start_date="2000-01-01",end_date="2024-01-01")

In [30]:
ng1_df = pull_TVdata(ticker="NG1!",exchange="NYMEX",interval=Interval.in_daily,start_date="2000-01-01",end_date="2024-01-01")

In [31]:
output_dir = '../data/raw-data'
bp_df.to_csv(os.path.join(output_dir,"BP_price.csv"), index=True)
shell_df.to_csv(os.path.join(output_dir,"SHEL_price.csv"), index=True)
oxy_df.to_csv(os.path.join(output_dir,"OXY_price.csv"), index=True)
xom_df.to_csv(os.path.join(output_dir,"XOM_price.csv"), index=True)
cvx_df.to_csv(os.path.join(output_dir,"CVX_price.csv"), index=True)
eqnr_df.to_csv(os.path.join(output_dir,"EQNR_price.csv"), index=True)
cl1_df.to_csv(os.path.join(output_dir,"CL1_price.csv"), index=True)
ng1_df.to_csv(os.path.join(output_dir,"NG1_price.csv"), index=True)